In [18]:
import os
import json
import boto3
from botocore.config import Config

# Lambda client
lambda_client = boto3.client("lambda")
cost_center = "abc"

# Bedrock client
bedrock = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
    config=Config(retries={"max_attempts": 3, "mode": "adaptive"}),
)



def prepare_bedrock_payload(event_body):
    return json.dumps({"prompt": event_body["inputs"], **event_body["parameters"]})


def invoke_bedrock(payload, model_id):
    response = bedrock.invoke_model(
        body=payload,
        modelId=model_id,
        accept="application/json",
        contentType="application/json",
    )
    print(response)
    if response["ResponseMetadata"]["HTTPStatusCode"] == 200:
        # Get text reponse from the model
        completion = json.loads(response.get("body").read())["completion"]
        request_id = response['ResponseMetadata']['RequestId']
        
        return completion, request_id

    else:
        raise Exception(
            "Bedrock API call failed with status "
            + str(response["ResponseMetadata"]["HTTPStatusCode"])
        )

In [19]:
event = {'resource': '/', 'path': '/', 'httpMethod': 'POST', 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'CloudFront-Forwarded-Proto': 'https', 'CloudFront-Is-Desktop-Viewer': 'true', 'CloudFront-Is-Mobile-Viewer': 'false', 'CloudFront-Is-SmartTV-Viewer': 'false', 'CloudFront-Is-Tablet-Viewer': 'false', 'CloudFront-Viewer-ASN': '12874', 'CloudFront-Viewer-Country': 'IT', 'Content-Type': 'application/json', 'Host': 'onlrex5fw0.execute-api.us-east-1.amazonaws.com', 'model_id': 'anthropic.claude-v2', 'User-Agent': 'python-requests/2.28.1', 'Via': '1.1 7c46840615f4a9403945d2fc5f3f0502.cloudfront.net (CloudFront)', 'X-Amz-Cf-Id': 'zTxGWpMQV8pYEFJKxwfPfEO9zJV7731FKVhmxEiK8NJSjCmJk7dpjw==', 'X-Amzn-Trace-Id': 'Root=1-650db8e5-675b3e7b411e4f032bb132ea', 'X-Forwarded-For': '93.42.47.3, 18.68.25.179', 'X-Forwarded-Port': '443', 'X-Forwarded-Proto': 'https'}, 'multiValueHeaders': {'Accept': ['*/*'], 'Accept-Encoding': ['gzip, deflate'], 'CloudFront-Forwarded-Proto': ['https'], 'CloudFront-Is-Desktop-Viewer': ['true'], 'CloudFront-Is-Mobile-Viewer': ['false'], 'CloudFront-Is-SmartTV-Viewer': ['false'], 'CloudFront-Is-Tablet-Viewer': ['false'], 'CloudFront-Viewer-ASN': ['12874'], 'CloudFront-Viewer-Country': ['IT'], 'Content-Type': ['application/json'], 'Host': ['onlrex5fw0.execute-api.us-east-1.amazonaws.com'], 'model_id': ['anthropic.claude-v2'], 'User-Agent': ['python-requests/2.28.1'], 'Via': ['1.1 7c46840615f4a9403945d2fc5f3f0502.cloudfront.net (CloudFront)'], 'X-Amz-Cf-Id': ['zTxGWpMQV8pYEFJKxwfPfEO9zJV7731FKVhmxEiK8NJSjCmJk7dpjw=='], 'X-Amzn-Trace-Id': ['Root=1-650db8e5-675b3e7b411e4f032bb132ea'], 'X-Forwarded-For': ['93.42.47.3, 18.68.25.179'], 'X-Forwarded-Port': ['443'], 'X-Forwarded-Proto': ['https']}, 'queryStringParameters': None, 'multiValueQueryStringParameters': None, 'pathParameters': None, 'stageVariables': None, 'requestContext': {'resourceId': 'cgfv64kxn8', 'resourcePath': '/', 'httpMethod': 'POST', 'extendedRequestId': 'LqnT7Ff9IAMFW8w=', 'requestTime': '22/Sep/2023:15:55:17 +0000', 'path': '/prod/', 'accountId': '232922378323', 'protocol': 'HTTP/1.1', 'stage': 'prod', 'domainPrefix': 'onlrex5fw0', 'requestTimeEpoch': 1695398117644, 'requestId': '395b24e5-8323-4782-8ec5-87bdffe36465', 'identity': {'cognitoIdentityPoolId': None, 'accountId': None, 'cognitoIdentityId': None, 'caller': None, 'sourceIp': '93.42.47.3', 'principalOrgId': None, 'accessKey': None, 'cognitoAuthenticationType': None, 'cognitoAuthenticationProvider': None, 'userArn': None, 'userAgent': 'python-requests/2.28.1', 'user': None}, 'domainName': 'onlrex5fw0.execute-api.us-east-1.amazonaws.com', 'apiId': 'onlrex5fw0'}, 'body': '{"inputs": "\\nHuman: What day comes after Tuesday?\\n\\nAssistant:\\n", "parameters": {"temperature": 0.2, "max_tokens_to_sample": 4096}}', 'isBase64Encoded': False}

In [20]:

bedrock_payload = prepare_bedrock_payload(json.loads(event["body"]))
model_id = event["headers"]["model_id"]

completion, request_id = invoke_bedrock(
    payload=bedrock_payload,
    model_id=model_id,
)

print(request_id)

logger.info(
    {
        "cost_center": cost_center,
        "request_id": request_id
    }
)


{'ResponseMetadata': {'RequestId': '2107cb20-ff9a-4069-bc78-20067f7c51fe', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 29 Sep 2023 12:50:28 GMT', 'content-type': 'application/json', 'content-length': '78', 'connection': 'keep-alive', 'x-amzn-requestid': '2107cb20-ff9a-4069-bc78-20067f7c51fe'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'body': <botocore.response.StreamingBody object at 0x7f8438193640>}
2107cb20-ff9a-4069-bc78-20067f7c51fe
